In [2]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.is_available())

cuda
True


In [5]:


# Đường dẫn đến tệp .pt đã lưu
model_path = './model/pretrained_bert_model_1re.pt'

# Khởi tạo mô hình BERT cho phân loại chuỗi và tải trạng thái từ tệp đã lưu
load_model = torch.load(model_path)

load_model.to(device)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

label_dict = {'sad': 2,
              'worry': 3,
              'so sad': 1,
              'happy': 4,
              'so happy': 5}
def predict_emotion(user_input):
    # Tokenize và chuẩn bị đầu vào từ người dùng
    inputs = tokenizer.encode_plus(
        user_input,
        add_special_tokens=True,
        max_length=64,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )

    # Di chuyển đầu vào đến thiết bị phù hợp nếu sử dụng GPU
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Sử dụng mô hình để dự đoán
    load_model.eval()
    with torch.no_grad():
        outputs = load_model(**inputs)

    # Lấy xác suất của lớp dự đoán
    class_names = ['Neutral', *list(label_dict.keys())]
    predicted_probabilities = outputs.logits.softmax(dim=1)
    predicted_class = torch.argmax(predicted_probabilities).item()
    print(predicted_class)
    predicted_class_name = class_names[predicted_class]
    print("Predicted Class:", predicted_class_name)

    # Trả về xác suất dự đoán
    return predicted_probabilities

In [7]:
# Gọi hàm dự đoán với đầu vào từ người dùng
user_input = "service is so awful"
predictions = predict_emotion(user_input)

# In xác suất dự đoán
print("Predicted Probabilities:", predictions)

1
Predicted Class: sad
Predicted Probabilities: tensor([[0.0158, 0.8276, 0.1207, 0.0125, 0.0111, 0.0124]])
